In [8]:
import boto3

# AWS credentials and region
aws_access_key = "YOUR_AWS_ACCESS_KEY"
aws_secret_key = "YOUR_AWS_SECRET_KEY"
region = "ap-south-1"  # Change this to your preferred region

# Create an EC2 client
ec2 = boto3.client('ec2', region_name=region,
                   aws_access_key_id=aws_access_key,
                   aws_secret_access_key=aws_secret_key)

# Create an EBS client
ebs = boto3.client('ec2', region_name=region,
                   aws_access_key_id=aws_access_key,
                   aws_secret_access_key=aws_secret_key)

# Launch an EC2 instance
instance = ec2.run_instances(
    ImageId='ami-0b9ecf71fe947bbdd',  # Replace with your preferred AMI ID
    InstanceType='t2.micro',
    MinCount=1,
    MaxCount=1
)

instance_id = instance['Instances'][0]['InstanceId']

# Wait for the instance to be running
ec2.get_waiter('instance_running').wait(InstanceIds=[instance_id])
print(f"EC2 instance with ID {instance_id} is now running.")

# Create an EBS volume
volume = ebs.create_volume(
    AvailabilityZone=f"{region}b",  # Replace 'b' with the desired availability zone (e.g., 'a', 'b', 'c')
    Size=8,
    VolumeType='gp2'
)

volume_id = volume['VolumeId']

# Wait for the EBS volume to be available
ebs.get_waiter('volume_available').wait(VolumeIds=[volume_id])
print(f"EBS volume with ID {volume_id} is now available.")

# Attach the EBS volume to the EC2 instance
response = ec2.attach_volume(
    Device='/dev/sdf',  # Replace with the desired device name (e.g., /dev/sdf)
    InstanceId=instance_id,
    VolumeId=volume_id
)

print(f"EBS volume with ID {volume_id} attached to instance {instance_id} as {response['Device']}.")


EC2 instance with ID i-026413277b5c4e8a7 is now running.
EBS volume with ID vol-02dc29828c4daa753 is now available.
EBS volume with ID vol-02dc29828c4daa753 attached to instance i-026413277b5c4e8a7 as /dev/sdf.
